In [1]:
!pip install pefile
!pip install numpy
!pip install sklearn
!pip install lightgbm
!pip install tqdm
!pip install pandas
!pip install graphviz
!pip install overload
!pip install lief
!pip install numpy
!pip install sklearn
!pip install tqdm
!pip install graphviz
!pip install lightgbm
#peminer와 ember 필요한 패키지 다운로드

     |████████████████████████████████| 71kB 3.4MB/s 
  Created wheel for pefile: filename=pefile-2019.4.18-cp36-none-any.whl size=60823 sha256=c9a67097e3eec6069a0916b89609deaca3108add61666dbca0217f3403c0ae4d
  Stored in directory: /root/.cache/pip/wheels/1c/a1/95/4f33011a0c013c872fe6f0f364dc463a2588120820e40a30d8
Successfully built pefile
  Created wheel for overload: filename=overload-1.1-cp36-none-any.whl size=5695 sha256=d6d4062d18b6c3010480a81dce2ee4a4e8f530a86180263eabf971d44e805bc0
  Stored in directory: /root/.cache/pip/wheels/bd/bd/06/69680a1b3ad39f87369affc3e999eb0773ffddbb9b50533d61
Successfully built overload
     |████████████████████████████████| 3.5MB 5.6MB/s 
ERROR: Operation cancelled by user


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!unzip /content/drive/My\ Drive/데이터.zip

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: PESTUDIO/학습데이터/bd9cf98f928470d06332223ddd84b0ac9af055a54a07775b75a497b4cc947919.json  
  inflating: PESTUDIO/학습데이터/bda2ab110f80f6fa79b3d650b6ae4565d8e840d7ba5b69e38c2874f63cc423a8.json  
  inflating: PESTUDIO/학습데이터/bda37d4ed18f33df81f03823e62707da458120a021d0878d6c450b5c0f157f3f.json  
  inflating: PESTUDIO/학습데이터/bdaa46116f28aeae7c99d9d18a06d9ed3181ed82a6e719fb0956e9672c2012cf.json  
  inflating: PESTUDIO/학습데이터/bdab51b550107ca68011548a4a84aae5115dcf9d9210e91cd9bfde15406a4a64.json  
  inflating: PESTUDIO/학습데이터/bdaf361591e90d89a9adbb3d070f27635029b705fcbbd99f7cfed738262e0d2b.json  
  inflating: PESTUDIO/학습데이터/bdb0b50f0281fedb6477b5202e460c0e6613a650f5352f5ddc4aa6ce5cb570b5.json  
  inflating: PESTUDIO/학습데이터/bdb251111c8b6ee237092544abbcc81daa8c6e237459bd497fe8f840f6d24d12.json  
  inflating: PESTUDIO/학습데이터/bdb46fc3b901bbecaf104504ea723df0512f77d037343c91e6db0f731c00d048.json  
  inflating: PESTUDIO/학습데이터/bdb60ce6b039af2a7842b56c24dcd5a88d32

In [4]:
import glob
import os
import pickle
import csv
import json
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import zipfile
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

from pprint import pprint
from tqdm.notebook import tqdm

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import confusion_matrix

from lightgbm import LGBMClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.feature_extraction import FeatureHasher
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import RFE
from sklearn.feature_extraction import DictVectorizer

from overload import overload

from sklearn import tree

import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [5]:
SEED = 41

def read_label_csv(path):
    label_table = dict()
    with open(path, "r",encoding='cp949') as f:
        for line in f.readlines()[1:]:
            fname, label = line.strip().split(",")
            label_table[fname] = int(label)
    return label_table

def read_json(path):
    with open(path, "r",encoding='cp949') as f:
        return json.load(f)

def load_model(**kwargs):
    if kwargs["model"] == "rf":
        return RandomForestClassifier(random_state=kwargs["random_state"], n_jobs=4)
    elif kwargs["model"] == "dt":
        return DecisionTreeClassifier(random_state=kwargs["random_state"])
    elif kwargs["model"] == "lgb":
        return LGBMClassifier(random_state=kwargs["random_state"])
    elif kwargs["model"] == "svm":
        return SVC(random_state=kwargs["random_state"])
    elif kwargs["model"] == "lr":
        return LogisticRegression(random_state=kwargs["random_state"], n_jobs=-1)
    elif kwargs["model"] == "knn":
        return KNeighborsClassifier(n_jobs=-1)
    elif kwargs["model"] == "adaboost":
        return AdaBoostClassifier(random_state=kwargs["random_state"])
    elif kwargs["model"] == "mlp":
        return MLPClassifier(random_state=kwargs["random_state"])
    else:
        print("Unsupported Algorithm")
        return None
    

def train(X_train, y_train, model):
    '''
        머신러닝 모델을 선택하여 학습을 진행하는 함수
	
        :param X_train: 학습할 2차원 리스트 특징벡터
        :param y_train: 학습할 1차원 리스트 레이블 벡터
        :param model: 문자열, 선택할 머신러닝 알고리즘
        :return: 학습된 머신러닝 모델 객체
    '''
    clf = load_model(model=model, random_state=SEED)
    clf.fit(X_train, y_train)
    return clf


def evaluate(X_test, y_test, model):
    '''
        학습된 머신러닝 모델로 검증 데이터를 검증하는 함수
	
        :param X_test: 검증할 2차원 리스트 특징 벡터
        :param y_test: 검증할 1차원 리스트 레이블 벡터
        :param model: 학습된 머신러닝 모델 객체
    '''
    predict = model.predict(X_test)
    print("정확도", model.score(X_test, y_test))


In [6]:
class PeminerParser:
    def __init__(self, path):
        self.report = read_json(path)
        self.vector = []
    
    def process_report(self):
        '''
            전체 데이터 사용        
        '''
        
        self.vector = [value for _, value in sorted(self.report.items(), key=lambda x: x[0])]
        return self.vector
    

class EmberParser:
    '''
        예제에서 사용하지 않은 특징도 사용하여 벡터화 할 것을 권장
    '''
    def __init__(self, path):
        self.report = read_json(path)
        self.vector = []
    
    def get_histogram_info(self):
        histogram = np.array(self.report["histogram"])
        total = histogram.sum()
        vector = histogram / total
        return vector.tolist()

    def get_byteentropy_info(self):
        byteentropy = np.array(self.report["byteentropy"])
        total = byteentropy.sum()
        vector = byteentropy / total
        return vector.tolist()
    
    def get_string_info(self):
        strings = self.report["strings"]

        hist_divisor = float(strings['printables']) if strings['printables'] > 0 else 1.0
        vector = [
            strings['numstrings'], 
            strings['avlength'], 
            strings['printables'],
            strings['entropy'], 
            strings['paths'], 
            strings['urls'],
            strings['registry'], 
            strings['MZ']
        ]
        vector += (np.asarray(strings['printabledist']) / hist_divisor).tolist()
        return vector
    
    def get_general_file_info(self):
        general = self.report["general"]
        vector = [
            general['size'], general['vsize'], general['has_debug'], general['exports'], general['imports'],
            general['has_relocations'], general['has_resources'], general['has_signature'], general['has_tls'],
            general['symbols']
        ]
        return vector

    def get_datadirectories_info(self):
        datadirectories = self.report["datadirectories"]
        vector = []
        if datadirectories == []:
          vector = [0] * 30
        else:
          for i in range(len(datadirectories)):
            vector += datadirectories[i]["size"], datadirectories[i]["virtual_address"]
        return vector

    def get_sections_info(self):
        raw_obj = self.report["section"]
        sections = raw_obj["sections"]
        general = [
            len(sections),  # total number of sections
            # number of sections with nonzero size
            sum(1 for s in sections if s['size'] == 0),
            # number of sections with an empty name
            sum(1 for s in sections if s['name'] == ""),
            # number of RX
            sum(1 for s in sections if 'MEM_READ' in s['props'] and 'MEM_EXECUTE' in s['props']),
            # number of W
            sum(1 for s in sections if 'MEM_WRITE' in s['props'])
        ]
        # gross characteristics of each section
        section_sizes = [(s['name'], s['size']) for s in sections]
        section_sizes_hashed = FeatureHasher(50, input_type="pair").transform([section_sizes]).toarray()[0]
        section_entropy = [(s['name'], s['entropy']) for s in sections]
        section_entropy_hashed = FeatureHasher(50, input_type="pair").transform([section_entropy]).toarray()[0]
        section_vsize = [(s['name'], s['vsize']) for s in sections]
        section_vsize_hashed = FeatureHasher(50, input_type="pair").transform([section_vsize]).toarray()[0]
        entry_name_hashed = FeatureHasher(50, input_type="string").transform([raw_obj['entry']]).toarray()[0]
        characteristics = [p for s in sections for p in s['props'] if s['name'] == raw_obj['entry']]
        characteristics_hashed = FeatureHasher(50, input_type="string").transform([characteristics]).toarray()[0]

        return np.hstack([
            general, section_sizes_hashed, section_entropy_hashed, section_vsize_hashed, entry_name_hashed,
            characteristics_hashed
        ]).astype(np.float32).tolist()

    def get_header_info(self):
        raw_obj = self.report["header"]
        return np.hstack([
            raw_obj['coff']['timestamp'],
            FeatureHasher(10, input_type="string").transform([[raw_obj['coff']['machine']]]).toarray()[0],
            FeatureHasher(10, input_type="string").transform([raw_obj['coff']['characteristics']]).toarray()[0],
            FeatureHasher(10, input_type="string").transform([[raw_obj['optional']['subsystem']]]).toarray()[0],
            FeatureHasher(10, input_type="string").transform([raw_obj['optional']['dll_characteristics']]).toarray()[0],
            FeatureHasher(10, input_type="string").transform([[raw_obj['optional']['magic']]]).toarray()[0],
            raw_obj['optional']['major_image_version'],
            raw_obj['optional']['minor_image_version'],
            raw_obj['optional']['major_linker_version'],
            raw_obj['optional']['minor_linker_version'],
            raw_obj['optional']['major_operating_system_version'],
            raw_obj['optional']['minor_operating_system_version'],
            raw_obj['optional']['major_subsystem_version'],
            raw_obj['optional']['minor_subsystem_version'],
            raw_obj['optional']['sizeof_code'],
            raw_obj['optional']['sizeof_headers'],
            raw_obj['optional']['sizeof_heap_commit'],
        ]).astype(np.float32).tolist()

    def get_imports_info(self):
        # unique libraries
        raw_obj = self.report["imports"]
        libraries = list(set([l.lower() for l in raw_obj.keys()]))
        libraries_hashed = FeatureHasher(256, input_type="string").transform([libraries]).toarray()[0]

        # A string like "kernel32.dll:CreateFileMappingA" for each imported function
        imports = [lib.lower() + ':' + e for lib, elist in raw_obj.items() for e in elist]
        imports_hashed = FeatureHasher(1024, input_type="string").transform([imports]).toarray()[0]

        # Two separate elements: libraries (alone) and fully-qualified names of imported functions
        return np.hstack([libraries_hashed, imports_hashed]).astype(np.float32).tolist()

    def get_exports_info(self):
        raw_obj = self.report["exports"]
        exports_hashed = FeatureHasher(128, input_type="string").transform([raw_obj]).toarray()[0]
        return exports_hashed.astype(np.float32).tolist()

    def process_report(self):
        vector = []
        vector += self.get_general_file_info()
        vector += self.get_histogram_info()
        vector += self.get_datadirectories_info()
        vector += self.get_string_info()
        vector += self.get_sections_info()
        vector += self.get_byteentropy_info()
        vector += self.get_header_info()
        vector += self.get_imports_info()
        vector += self.get_exports_info()


        '''
            특징 추가
        '''
        return vector
    
    
class PestudioParser:
    '''
        사용할 특징을 선택하여 벡터화 할 것을 권장
    '''
    
    def __init__(self, path):
        self.report = read_json(path)
        self.vector = []
    def overview(self) :
          size = float(self.report['image']['overview']['size']) / 10000
          entropy = float(self.report['image']['overview']['entropy'])
          vector = [size,entropy]
          return vector
    def indicator(self) :
        vector_hashed = []
        if len(self.report['image']) < 20 :
            vector_hashed = [-1 for x in range(50)]
        else :
          if len(self.report['image']['indicators']) !=1 :
            vector = [instance for instance in self.report['image']['indicators']['indicator']]
            vector_hashed = FeatureHasher(50).transform(vector).toarray()[0].astype(np.float32).tolist()
          else :
            vector = [self.report['image']['indicators']['indicator']]
            print(vector)
            vector_hashed = FeatureHasher(50).transform(vector).toarray()[0].astype(np.float32).tolist()
        return vector_hashed
    def virustotal(self) :
        if type(self.report['image']['virustotal']) != dict :
           vector = [-1]
        else :
           vector = [float(self.report['image']['virustotal']['@detection'])]
        return vector
    def dos(self) :
        if len(self.report['image']) < 20 :
          vector = [-1,-1,-1,-1,-1,-1]
        else :
          vector = [
                   float(self.report['image']['dos-header']['entropy']),
                   float(self.report['image']['dos-header']['size']),
                   float(self.report['image']['dos-stub']['entropy']),
                   float(self.report['image']['dos-stub']['size']),
                   float(self.report['image']['file-header']['size-of-optional-header']),
                   float(self.report['image']['optional-header']['size-of-headers'])
                   ]
        return vector
    def resource_extraction(self) :
      if len(self.report['image']['resources'])==2 :
        if len(self.report['image']['resources']['instance']) !=10 :
           vector = [instance for instance in self.report['image']['resources']['instance']]
           vector_hashed = FeatureHasher(51).transform(vector).toarray()[0].astype(np.float32).tolist()
        else :
           vector = [self.report['image']['resources']['instance']]
           vector_hashed = FeatureHasher(51).transform(vector).toarray()[0].astype(np.float32).tolist()
      else :
           vector_hashed = [-1 for x in range(51)]
      return vector_hashed
    def process_report(self) :
        vector = []
        vector += self.overview()
        vector += self.indicator()
        vector += self.virustotal()
        vector += self.dos()
        vector += self.resource_extraction()
        return vector

{
    "image": {
        "overview": {
            "@name": "f:\\ts\\data\\kisa2018\\\ud559\uc2b5\ub370\uc774\ud130\\6754fbf434fe75d487a9095784cef4ca343ae224c59b2beb43c37ea46cbc9360.vir",
            "description": "n/a",
            "file-version": "n/a",
            "file-type": "dynamic-link-library",
            "cpu": "32",
            "size": "67584",
            "size-without-overlay": "n/a",
            "subsystem": "GUI",
            "signature": "n/a",
            "entropy": "7.096",
            "compiler-stamp": "Thu Jul 26 20:45:18 2012",
            "debugger-stamp": "Thu Jul 26 20:45:18 2012",
            "resources-stamp": "empty",
            "exports-stamp": "Sat Apr 16 05:06:26 2016",
            "version-stamp": "n/a",
            "entry-point": "60 6A 00 31 2C 24 89 E5 8D 65 B0 BA 17 00 00 00 B8 01 00 00 00 83 C0 D2 BF 01 00 00 00 01 C7 0B 3D",
            "first-bytes-hex": "4D 5A 90 00 03 00 00 00 04 00 00 00 FF FF 00 00 B8 00 00 00 00 00 00 00 40 00 00 00 00 00 0

In [ ]:
V, w = [], []
x=0
label_table = read_label_csv("학습데이터_정답.csv")
for fname in os.listdir("/content/PEMINER/학습데이터"):
    feature_vector = []
    lname = fname.split('.')[0]
    # if lname == '' :
    #   feature_vector = [-1 for x in range(668)]
    #   label = 0
    # else :
    label = label_table[lname]
    for data in ["/content/PEMINER/학습데이터", "/content/EMBER/학습데이터","/content/PESTUDIO/학습데이터"]:
          path = f"{data}/{fname}"
          if data == "/content/PEMINER/학습데이터":
              feature_vector += PeminerParser(path).process_report()

          elif data == "/content/EMBER/학습데이터":
              feature_vector += EmberParser(path).process_report()
    
          else:
            try :
              feature_vector += PestudioParser(path).process_report()
            except :
              feature_vector += [-1 for x in range(110)]
    V.append(feature_vector)
    w.append(label)
    x = x+1
    print(format(x),"번째 완료")


np.asarray(V).shape, np.asarray(w).shape

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
15001 번째 완료
15002 번째 완료
15003 번째 완료
15004 번째 완료
15005 번째 완료
15006 번째 완료
15007 번째 완료
15008 번째 완료
15009 번째 완료
15010 번째 완료
15011 번째 완료
15012 번째 완료
15013 번째 완료
15014 번째 완료
15015 번째 완료
15016 번째 완료
15017 번째 완료
15018 번째 완료
15019 번째 완료
15020 번째 완료
15021 번째 완료
15022 번째 완료
15023 번째 완료
15024 번째 완료
15025 번째 완료
15026 번째 완료
15027 번째 완료
15028 번째 완료
15029 번째 완료
15030 번째 완료
15031 번째 완료
15032 번째 완료
15033 번째 완료
15034 번째 완료
15035 번째 완료
15036 번째 완료
15037 번째 완료
15038 번째 완료
15039 번째 완료
15040 번째 완료
15041 번째 완료
15042 번째 완료
15043 번째 완료
15044 번째 완료
15045 번째 완료
15046 번째 완료
15047 번째 완료
15048 번째 완료
15049 번째 완료
15050 번째 완료
15051 번째 완료
15052 번째 완료
15053 번째 완료
15054 번째 완료
15055 번째 완료
15056 번째 완료
15057 번째 완료
15058 번째 완료
15059 번째 완료
15060 번째 완료
15061 번째 완료
15062 번째 완료
15063 번째 완료
15064 번째 완료
15065 번째 완료
15066 번째 완료
15067 번째 완료
15068 번째 완료
15069 번째 완료
15070 번째 완료
15071 번째 완료
15072 번째 완료
15073 번째 완료
15074 번째 완료
15075 번째 완료
15076 번째 완료
15077 번째 완료
15078 번째 완료
15079 번째 완료
15080 번째 완료
1508

((20000, 2679), (20000,))

In [ ]:
X, y = [], []
x = 0
label_table = read_label_csv("검증데이터_정답.csv")

for fname in os.listdir("/content/PEMINER/검증데이터"):
    feature_vector = []
    lname = fname.split('.')[0]
    # if lname == '' :
    #   feature_vector = [-1 for x in range(668)]
    #   label = 0
    # else :
    label = label_table[lname]
    for data in ["/content/PEMINER/검증데이터", "/content/EMBER/검증데이터","/content/PESTUDIO/검증데이터"]:
          path = f"{data}/{fname}"
          if data == "/content/PEMINER/검증데이터":
              feature_vector += PeminerParser(path).process_report()
          elif data == "/content/EMBER/검증데이터":
              feature_vector += EmberParser(path).process_report()
          else:
            try :
              feature_vector += PestudioParser(path).process_report()
            except :
              feature_vector += [-1 for x in range(110)]
    X.append(feature_vector)
    y.append(label)
    x = x+1
    print(format(x),"번째 완료")

np.asarray(X).shape, np.asarray(y).shape

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
5001 번째 완료
5002 번째 완료
5003 번째 완료
5004 번째 완료
5005 번째 완료
5006 번째 완료
5007 번째 완료
5008 번째 완료
5009 번째 완료
5010 번째 완료
5011 번째 완료
5012 번째 완료
5013 번째 완료
5014 번째 완료
5015 번째 완료
5016 번째 완료
5017 번째 완료
5018 번째 완료
5019 번째 완료
5020 번째 완료
5021 번째 완료
5022 번째 완료
5023 번째 완료
5024 번째 완료
5025 번째 완료
5026 번째 완료
5027 번째 완료
5028 번째 완료
5029 번째 완료
5030 번째 완료
5031 번째 완료
5032 번째 완료
5033 번째 완료
5034 번째 완료
5035 번째 완료
5036 번째 완료
5037 번째 완료
5038 번째 완료
5039 번째 완료
5040 번째 완료
5041 번째 완료
5042 번째 완료
5043 번째 완료
5044 번째 완료
5045 번째 완료
5046 번째 완료
5047 번째 완료
5048 번째 완료
5049 번째 완료
5050 번째 완료
5051 번째 완료
5052 번째 완료
5053 번째 완료
5054 번째 완료
5055 번째 완료
5056 번째 완료
5057 번째 완료
5058 번째 완료
5059 번째 완료
5060 번째 완료
5061 번째 완료
5062 번째 완료
5063 번째 완료
5064 번째 완료
5065 번째 완료
5066 번째 완료
5067 번째 완료
5068 번째 완료
5069 번째 완료
5070 번째 완료
5071 번째 완료
5072 번째 완료
5073 번째 완료
5074 번째 완료
5075 번째 완료
5076 번째 완료
5077 번째 완료
5078 번째 완료
5079 번째 완료
5080 번째 완료
5081 번째 완료
5082 번째 완료
5083 번째 완료
5084 번째 완료
5085 번째 완료
5086 번째 완료
5087 번째 완료
5088 번째

((10000, 2679), (10000,))

In [ ]:
# 학습
models = []
for model in ["rf"]:
    clf = train(V, w, model)
    models.append(clf)
   

# 검증
# 실제 검증 시에는 제공한 검증데이터를 검증에 사용해야 함
for model in models:
    evaluate(X, y, model)


정확도 0.9486
100
None
1
2
[1 0 1 ... 1 1 1]


In [ ]:
def ensemble_result(X, y, models):
    '''
        학습된 모델들의 결과를 앙상블하는 함수
	
        :param X: 검증할 2차원 리스트 특징 벡터
        :param y: 검증할 1차원 리스트 레이블 벡터
        :param models: 1개 이상의 학습된 머신러닝 모델 객체를 가지는 1차원 리스트
    '''
    # Soft Voting
    # https://devkor.tistory.com/entry/Soft-Voting-%EA%B3%BC-Hard-Voting
    predicts = []
    for model in models:
        prob = [result for _, result in model.predict_proba(X)]
        predicts.append(prob)
    
    predict = np.mean(predicts, axis=0)
    predict = [1 if x >= 0.5 else 0 for x in predict]
        
    print("정확도", accuracy_score(y, predict))
ensemble_result(V,w,models)

정확도 0.99945


In [ ]:
parameters = { 'n_estimators' : [100],
           'max_depth' : [6, 8, 10, 12],
           'min_samples_leaf' : [8, 12, 18],
           'min_samples_split' : [8, 16, 20]
            }
rf_clf = RandomForestClassifier(random_state=SEED)
grid_cv = GridSearchCV(rf_clf, param_grid=parameters, cv=3, n_jobs=-1)

grid_cv.fit(V, w)

print('최적 하이퍼 파라미터: ', grid_cv.best_params_)
print('최고 예측 정확도: {:.4f}'.format(grid_cv.best_score_))

최적 하이퍼 파라미터:  {'max_depth': 12, 'min_samples_leaf': 8, 'min_samples_split': 20, 'n_estimators': 100}
최고 예측 정확도: 0.9402


In [ ]:
ftr_importances_values = rf_clf1.feature_importances_
ftr_importances = pd.Series(ftr_importances_values, index = X_train.columns)
ftr_top20 = ftr_importances.sort_values(ascending=False)[:20]

plt.figure(figsize=(8,6))
plt.title('Top 20 Feature Importances')
sns.barplot(x=ftr_top20, y=ftr_top20.index)
plt.show()

AttributeError: ignored

In [ ]:
def select_feature(X, y, model):
    '''
        주어진 특징 벡터에서 특정 알고리즘 기반 특징 선택
        
        본 예제에서는 RFE 알고리즘 사용
      c
        
        :param X: 검증할 2차원 리스트 특징 벡터
        :param y: 검증할 1차원 리스트 레이블 벡터
        :param model: 문자열, 특징 선택에 사용할 머신러닝 알고리즘
    '''
    
    model = load_model(model=model, random_state=SEED)
    rfe = RFE(estimator=model)
    return rfe.fit_transform(X, y)

selected_X = select_feature(V, w, "rf")
new_model = train(selected_X, w, "rf")

In [ ]:
for model in new_model:
    evaluate(X, y, model)